In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="test")
location = geolocator.geocode(u"nashville")

print(location.address)       #住所
print(location.latitude)      #緯度
print(location.longitude)     #経度
print(location.raw)           #ジオコーディングした結果の全ての情報

Nashville-Davidson, Davidson County, Middle Tennessee, Tennessee, United States
36.1622767
-86.7742984
{'place_id': 298048751, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 197472, 'lat': '36.1622767', 'lon': '-86.7742984', 'class': 'boundary', 'type': 'administrative', 'place_rank': 16, 'importance': 0.6474694013099324, 'addresstype': 'city', 'name': 'Nashville-Davidson', 'display_name': 'Nashville-Davidson, Davidson County, Middle Tennessee, Tennessee, United States', 'boundingbox': ['35.9677850', '36.4054960', '-87.0547660', '-86.5155880']}


In [24]:
import unicodedata
from pathlib import Path
!pip install scikit-learn==1.3.0

import os

!pip install catboost
import catboost as cb
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
!pip install category_encoders
from category_encoders import CountEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import TargetEncoder

from geopy.geocoders import Nominatim
# geolocator = Nominatim()

from geopy.extra.rate_limiter import RateLimiter
import time

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
!pip install pytorch-tabnet
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

import torch.optim as optim

In [25]:
INPUT_DIR = '/content/drive/MyDrive/signate/Student Cup 2023/input'

df = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))
df

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,fwd,mid-size,SUV,blue,ny,16553
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,fwd,mid-size,sedan,red,ca,5158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,rwd,compact,truck,black,pa,32212
27528,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072,clean,automatic,rwd,full-size,pickup,black,NaN,5400
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,4wd,mid-size,SUV,white,ny,22227
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,4wd,mid-size,sedan,white,ny,3054


In [26]:
train_df = pd.read_csv(os.path.join(INPUT_DIR, "train.csv"))
test_df = pd.read_csv(os.path.join(INPUT_DIR, "test.csv"))
sub_df = pd.read_csv(os.path.join(INPUT_DIR, "submit_sample.csv"), names=["id", "price"])

In [27]:
for col in train_df.select_dtypes(include=['float', "int"]).columns:
    print(col)
    train_df[col].fillna(0, inplace=True)

for col in train_df.select_dtypes(exclude=['float', "int"]).columns:
    print(col)
    train_df[col].fillna("unknown", inplace=True)

for col in test_df.select_dtypes(include=['float', "int"]).columns:
    print(col)
    test_df[col].fillna(0, inplace=True)

for col in test_df.select_dtypes(exclude=['float', "int"]).columns:
    print(col)
    test_df[col].fillna("unknown", inplace=True)


id
year
odometer
price
region
manufacturer
condition
cylinders
fuel
title_status
transmission
drive
size
type
paint_color
state
id
year
odometer
region
manufacturer
condition
cylinders
fuel
title_status
transmission
drive
size
type
paint_color
state


In [28]:
train_df.isnull().sum()

id              0
region          0
year            0
manufacturer    0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
state           0
price           0
dtype: int64

In [29]:
test_df.isnull().sum()

id              0
region          0
year            0
manufacturer    0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            0
type            0
paint_color     0
state           0
dtype: int64

In [30]:
train_df['state'].unique()

array(['unknown', 'pa', 'ks', 'ny', 'ca', 'al', 'or', 'va', 'mt', 'nj',
       'ma', 'wi', 'sc', 'wa', 'dc', 'oh', 'in', 'de', 'fl', 'nm', 'az',
       'ok', 'mn', 'co', 'nv', 'wv', 'tn', 'mi', 'ri', 'il', 'tx', 'ut',
       'ia', 'ga', 'md', 'mo', 'ky', 'nc', 'ak', 'id', 'ct', 'wy', 'nd',
       'me', 'ar', 'hi', 'sd', 'ne', 'nh', 'vt', 'la', 'ms'], dtype=object)

In [31]:
# region -> stateが一意に定まることを確認
region_state = {region:{} for region in train_df['region'].unique()}
for row, value in train_df.iterrows():
    if not pd.isna(value['state']):
        if value['state'] not in region_state[value['region']]:
            region_state[value['region']][value['state']] = 1
        else:
            region_state[value['region']][value['state']] += 1

for region, state_dict in region_state.items():
    if len(state_dict) > 1 or state_dict == {}:
        region_state[region] = pd.NA
    else:
        region_state[region] = list(state_dict.keys())[0]

# regionからstateを決定
train_df['state'] = [region_state[region] if pd.isna(state) else state for region, state in zip(train_df['region'], train_df['state'])]
test_df['state'] = [region_state[region] if pd.isna(state) else state for region, state in zip(test_df['region'], test_df['state'])]

In [32]:
# yearが3000年以降のものはおかしいので2000年に置換
err_idx = train_df.query("year >= 3000").index
train_df.loc[err_idx, "year"] = train_df.loc[err_idx, "year"].apply(lambda x: x - 1000)

# 走行距離 / 製造年
train_df["odometer_per_year"] = train_df["odometer"] / (2023 - train_df["year"])

err_idx = test_df.query("year >= 3000").index
test_df.loc[err_idx, "year"] = test_df.loc[err_idx, "year"].apply(lambda x: x - 1000)

# 走行距離 / 製造年
test_df["odometer_per_year"] = test_df["odometer"] / (2023 - test_df["year"])

In [33]:
agg_cate_col = [
    # 'region',
    "cylinders",
    "manufacturer",
    "condition",
    "fuel",
    "title_status",
    "transmission",
    "drive",
    "size",
    "type",
    "paint_color",
    "state",
    # "country",
    # "color_category"
]

agg_num_col = [
    "odometer",
    "odometer_per_year",
    "year"
]
# manufacturer  paint_color transmission

for c in agg_cate_col:
  for n in agg_num_col:
    grouped = train_df.groupby(c)[n]
    agg_data = grouped.agg(
        **{
            f"{c}_median_{n}": 'median',
            f"{c}_mean_{n}": 'mean'
        }
    ).reset_index()

    # 元のデータフレームに計算結果をマージ
    train_df = train_df.merge(agg_data, on=c, how='left')
    test_df = test_df.merge(agg_data, on=c, how='left')

In [34]:
train_df

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,...,paint_color_median_odometer_per_year,paint_color_mean_odometer_per_year,paint_color_median_year,paint_color_mean_year,state_median_odometer,state_mean_odometer,state_median_odometer_per_year,state_mean_odometer_per_year,state_median_year,state_mean_year
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,...,4858.912500,5826.843929,2005.0,1999.493056,111971.0,117267.042373,8355.471660,9508.917514,2011.0,2007.840194
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,...,9434.600000,10441.247719,2011.0,2008.769535,113441.0,115971.451679,8602.714286,9478.326389,2011.0,2008.174975
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,...,9434.600000,10441.247719,2011.0,2008.769535,112107.5,117177.051587,8258.032967,8987.520673,2011.0,2006.507937
3,3,albany,2014,ford,excellent,4 cylinders,gas,104118,clean,manual,...,8602.000000,9876.921532,2012.0,2008.779283,104647.0,107273.615426,8329.333333,9376.408248,2012.0,2008.684954
4,4,redding,2005,ford,excellent,6 cylinders,gas,144554,clean,manual,...,8111.000000,9206.404291,2008.0,2005.959831,106447.0,109720.457439,8541.255411,9455.623152,2012.0,2008.787935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27527,27527,williamsport,2008,ford,good,6 cylinders,gas,26660,clean,automatic,...,8484.416667,9489.091770,2012.0,2009.199744,113441.0,115971.451679,8602.714286,9478.326389,2011.0,2008.174975
27528,27528,tulsa,2007,ford,excellent,8 cylinders,gas,108072,clean,automatic,...,8484.416667,9489.091770,2012.0,2009.199744,111971.0,117267.042373,8355.471660,9508.917514,2011.0,2007.840194
27529,27529,rochester,2019,jeep,like new,6 cylinders,gas,139908,clean,automatic,...,7712.700000,8536.478779,2011.0,2007.150938,104647.0,107273.615426,8329.333333,9376.408248,2012.0,2008.684954
27530,27530,rochester,2007,jeep,excellent,6 cylinders,gas,112326,clean,automatic,...,7712.700000,8536.478779,2011.0,2007.150938,104647.0,107273.615426,8329.333333,9376.408248,2012.0,2008.684954


In [35]:
class PreProcessTransformer(TransformerMixin, BaseEstimator):
  def fit(self, X, y=None):
    return self

  def transform(self, X):
    # X["title_status"] = X["title_status"].fillna("clean")

    # cylinderから数値を取り出す
    X["cylinders"] = X["cylinders"].astype(str).str.extract("(\d+)").astype("float32")
    # X["cylinders"] = X["cylinders"].astype(str).str.extract("(\d+)").fillna(0).astype("float32")


    # sizeの表記揺れを修正
    X["size"] = X["size"].str.replace("ー", "-").astype(str)
    X["size"] = X["size"].str.replace("−", "-").astype(str)

    # 順位付け: サイズ別に値を割り当てます。大きいほど高い値にします。
    ranking_dict = {'full-size': 4, 'mid-size': 3, 'compact': 2, 'sub-compact': 1}

    # 新しい順位カラムを作成します。
    X['size_rank'] = X['size'].map(ranking_dict)

    # manufacturerの表記揺れを修正
    X["manufacturer"] = X["manufacturer"].apply(
        lambda x: unicodedata.normalize("NFKC", x).lower()
    )

    # 正規化の辞書
    normalize_dict = {
        'niѕsan': 'nissan',
        'subαru': 'subaru',
        'toyotа': 'toyota',
        'sαturn': 'saturn',
        'аcura': 'acura',
        'vоlkswagen': 'volkswagen',
        'lexuѕ': 'lexus',
        'ᴄhrysler': 'chrysler'
    }


    # データフレームのメーカー名を正規化
    X['manufacturer'] = X['manufacturer'].map(lambda x: normalize_dict.get(x, x))

    # 国別分類
    def classify_country(manufacturer):
        if pd.isnull(manufacturer):
            return 'Unknown'
        if manufacturer in ['bmw', 'volkswagen', 'audi', 'mercedes-benz', 'porsche']:
            return 'Germany'
        elif manufacturer in ['toyota', 'nissan', 'lexus', 'subaru', 'honda', 'mazda', 'mitsubishi', 'infiniti', 'acura']:
            return 'Japan'
        elif manufacturer in ['ford', 'buick', 'ram', 'dodge', 'chevrolet', 'jeep', 'chrysler', 'gmc', 'lincoln', 'cadillac', 'pontiac', 'saturn']:
            return 'USA'
        elif manufacturer in ['hyundai', 'kia']:
            return 'South Korea'
        elif manufacturer in ['fiat', 'alfa-romeo']:
            return 'Italy'
        elif manufacturer in ['land rover', 'jaguar', 'aston-martin', 'rover']:
            return 'UK'
        else:
            return 'Unknown'

    X['country'] = X['manufacturer'].apply(classify_country)


    def classify_color(color):
        if color in ['orange', 'yellow', 'red', 'white']:
            return 'Bright'
        elif color in ['black', 'brown', 'green', 'blue']:
            return 'Dark'
        elif color in ['silver', 'grey']:
            return 'Neutral'
        elif color in ['purple', 'custom']:
            return 'Special'
        else:
            return 'Unknown'

    X['color_category'] = X['paint_color'].apply(classify_color)

    # one-hot encoding
    # X = pd.get_dummies(X, columns=['country'])

    # yearが3000年以降のものはおかしいので2000年に置換
    err_idx = X.query("year >= 3000").index
    X.loc[err_idx, "year"] = X.loc[err_idx, "year"].apply(lambda x: x - 1000)

    # 走行距離 / 製造年
    # X["odometer_per_year"] = X["odometer"] / (2023 - X["year"])

    for c in agg_cate_col:
      for n in agg_num_col:
        print(c, n)
        X[f"{c}_diff_{n}_mean"] = X[n] - X[f"{c}_mean_{n}"]
        X[f"{c}_diff_{n}_median"] = X[n] - X[f"{c}_median_{n}"]

    # X["pass_year"] = 2023 - X["year"]

    # X['state'] = X['state'].replace('<NA>', 'null')
    # X.dropna(subset=['state'], inplace=True)
    X['state'] = X['state'].fillna("unknown")

    err_idx = X.query("odometer < -1").index
    X.loc[err_idx, "odometer"] = X.loc[err_idx, "odometer"].apply(lambda x: -1)

    return X

class OriginalTransformer(TransformerMixin, BaseEstimator):
    """数値特徴はそのまま、カテゴリ特徴はcategory型に変換"""

    # def __init__(self, numeric_cols, categorical_cols):
    def __init__(self, numeric_cols):
        self.numeric_cols = numeric_cols
        # self.categorical_cols = categorical_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_new = X.copy()

        # 数値変数
        X_new[self.numeric_cols] = X_new[self.numeric_cols].astype("float32")

        # カテゴリ変数
        # X_new[self.categorical_cols] = X_new[self.categorical_cols].astype("category")

        return X_new[self.get_feature_names_out()]

    def get_feature_names_out(self):
        # return self.numeric_cols + self.categorical_cols
        return self.numeric_cols

In [36]:
agg_cols = []
for n in agg_num_col:
  agg_cols += [f"{c}_diff_{n}_{stat}" for c in agg_cate_col for stat in ['mean', 'median']]
  agg_cols += [f"{c}_{stat}_{n}" for c in agg_cate_col for stat in ['mean', 'median']]

print(agg_cols)

numeric_cols = ["year", "odometer", "odometer_per_year", "size_rank"] + agg_cols
categorical_cols = [
    'region',
    "cylinders",
    "manufacturer",
    "condition",
    "fuel",
    "title_status",
    "transmission",
    "drive",
    "size",
    "type",
    "paint_color",
    "state",
    "country",
    "color_category"
]

ct = ColumnTransformer(
    transformers=[
        (
            "ori",
            OriginalTransformer(numeric_cols),
            numeric_cols,
        ),
        (
            "tgt",
            TargetEncoder(target_type="continuous", random_state=88),
            [
                'region',
                "cylinders",
                "manufacturer",
                "condition",
                "fuel",
                "title_status",
                "transmission",
                "drive",
                "size",
                "type",
                "paint_color",
                "state",
                "country",
                "color_category"
            ],
        ),
        # *[
        #     (f"agg_{key}", AggTransformer(key, ["odometer"], {"mean"}), [key] + ["odometer"])
        #     for key in [
        #         "manufacturer",
        #     ]
        # ],
    ],
    verbose=True,
)
ct.set_output(transform="pandas")
from sklearn.preprocessing import StandardScaler

# 既存のパイプラインにStandardScalerを追加
preprocess_pipeline = Pipeline(
    steps=[
        ("preprocess", PreProcessTransformer()),  # 既存の前処理
        ("ct", ct),
        ("standardize", StandardScaler()),  # 標準化を追加

    ]
)

X_train_transformed = preprocess_pipeline.fit_transform(train_df.drop(columns=["price", "id"]), train_df["price"])
X_test_transformed = preprocess_pipeline.transform(test_df.drop(columns=["id"]))

# y_train = train_df["price"].values

# # for col in X_train_transformed.select_dtypes(include=['float', "int"]).columns:
# #     print(col)
# #     X_train_transformed[col].fillna(0, inplace=True)

# # for col in X_test_transformed.select_dtypes(include=['float', "int"]).columns:
# #     print(col)
# #     X_test_transformed[col].fillna(0, inplace=True)

# # Convert to PyTorch tensors
# X_train_tensor = torch.FloatTensor(X_train_transformed)
# y_train_tensor = torch.FloatTensor(y_train).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test_transformed)




['cylinders_diff_odometer_mean', 'cylinders_diff_odometer_median', 'manufacturer_diff_odometer_mean', 'manufacturer_diff_odometer_median', 'condition_diff_odometer_mean', 'condition_diff_odometer_median', 'fuel_diff_odometer_mean', 'fuel_diff_odometer_median', 'title_status_diff_odometer_mean', 'title_status_diff_odometer_median', 'transmission_diff_odometer_mean', 'transmission_diff_odometer_median', 'drive_diff_odometer_mean', 'drive_diff_odometer_median', 'size_diff_odometer_mean', 'size_diff_odometer_median', 'type_diff_odometer_mean', 'type_diff_odometer_median', 'paint_color_diff_odometer_mean', 'paint_color_diff_odometer_median', 'state_diff_odometer_mean', 'state_diff_odometer_median', 'cylinders_mean_odometer', 'cylinders_median_odometer', 'manufacturer_mean_odometer', 'manufacturer_median_odometer', 'condition_mean_odometer', 'condition_median_odometer', 'fuel_mean_odometer', 'fuel_median_odometer', 'title_status_mean_odometer', 'title_status_median_odometer', 'transmission_m

In [37]:
# X_train_transformed.isnull().sum()

In [38]:
import torch.nn as nn
import torch.nn.functional as F

class PricePredictor(nn.Module):
    def __init__(self, input_dim):
        super(PricePredictor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.dropout = nn.Dropout(0.5)  # ドロップアウトを追加

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # ドロップアウトを適用
        x = F.relu(self.fc2(x))
        x = self.dropout(x)  # ドロップアウトを適用
        x = F.relu(self.fc3(x))
        return self.fc4(x)


import torch

def mape_loss(output, target):
    """
    Compute Mean Absolute Percentage Error (MAPE)
    """
    epsilon = 1e-6  # 小さな値を追加して、ゼロでの除算を防ぐ
    abs_err = torch.abs(target - output)
    return torch.mean(abs_err / (torch.abs(target) + epsilon) * 100)

loss_function = mape_loss
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)


In [39]:
# 5-foldクロスバリデーションの設定
n_splits = 5

# トレーニングのループ
epochs = 64
batch_size = 32

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

X = preprocess_pipeline.fit_transform(train_df.drop(columns=["price", "id"]), train_df["price"])
y = train_df["price"].values

# 各foldでの結果を保存するためのリスト
train_losses = []
val_losses = []
all_predictions = []


for fold, (train_index, val_index) in enumerate(kf.split(X)):
    print(f"Fold {fold + 1}/{n_splits}")

    X_train_fold, X_val_fold = X[train_index], X[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    X_train_tensor = torch.FloatTensor(X_train_fold)
    y_train_tensor = torch.FloatTensor(y_train_fold).view(-1, 1)
    X_val_tensor = torch.FloatTensor(X_val_fold)
    y_val_tensor = torch.FloatTensor(y_val_fold).view(-1, 1)

    model = PricePredictor(X_train_tensor.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.9)

    for epoch in range(epochs):
        # Training
        model.train()
        epoch_train_losses = []
        for i in range(0, len(X_train_tensor), batch_size):
            X_batch = X_train_tensor[i:i+batch_size]
            y_batch = y_train_tensor[i:i+batch_size]

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_function(outputs, y_batch)
            loss.backward()
            optimizer.step()

            epoch_train_losses.append(loss.item())

        scheduler.step()

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val_tensor)
            val_loss = loss_function(val_outputs, y_val_tensor)

        avg_train_loss = sum(epoch_train_losses) / len(epoch_train_losses)

        if (epoch+1) % 20 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Training Loss: {avg_train_loss}, Validation Loss: {val_loss.item()}")

    train_losses.append(avg_train_loss)
    val_losses.append(val_loss.item())

print(f"Average Training Loss: {sum(train_losses)/n_splits}")
print(f"Average Validation Loss: {sum(val_losses)/n_splits}")


cylinders odometer
cylinders odometer_per_year
cylinders year
manufacturer odometer
manufacturer odometer_per_year
manufacturer year
condition odometer
condition odometer_per_year
condition year
fuel odometer
fuel odometer_per_year
fuel year
title_status odometer
title_status odometer_per_year
title_status year
transmission odometer
transmission odometer_per_year
transmission year
drive odometer
drive odometer_per_year
drive year
size odometer
size odometer_per_year
size year
type odometer
type odometer_per_year
type year
paint_color odometer
paint_color odometer_per_year
paint_color year
state odometer
state odometer_per_year
state year
[ColumnTransformer] ........... (1 of 2) Processing ori, total=   0.5s
[ColumnTransformer] ........... (2 of 2) Processing tgt, total=   0.2s
Fold 1/5
Epoch 20/64, Training Loss: 46.59083289435709, Validation Loss: 45.11988830566406
Epoch 40/64, Training Loss: 46.404088644573065, Validation Loss: 44.997833251953125
Epoch 60/64, Training Loss: 45.956737

In [40]:
all_predictions = []

for fold, (train_index, val_index) in enumerate(kf.split(X)):
    # ... [トレーニングのコード]

    # テストデータに対する予測
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor)
        all_predictions.append(predictions)

# 予測の平均を計算
average_predictions = torch.stack(all_predictions).mean(dim=0)

In [41]:
submission_df = test_df[["id"]].copy()
submission_df["price"] = average_predictions.numpy().flatten()
OUTPUT_DIR = '/content/drive/MyDrive/signate/Student Cup 2023/output'
submission_df.to_csv(os.path.join(OUTPUT_DIR, "submission-035-NN.csv"), index=False, header=False)
submission_df


,id,price
0,27532,8509.936523
1,27533,4795.858398
2,27534,4753.461914
3,27535,17414.195312
4,27536,3704.615234
...,...,...
27532,55064,14642.462891
27533,55065,11787.829102
27534,55066,7222.695312
27535,55067,4378.500977
